In [1]:
input_width = 160
input_height = 100
channels = 3
class_number = 12

In [2]:
from keras.models import load_model
import keras.backend as K
def actual_acc(y_true, y_pred):
    return K.equal(K.argmax(y_pred), K.argmax(y_true))

model = load_model('models/worm_single_large_adam.h5',custom_objects={'actual_acc':actual_acc})
model.summary()

Using TensorFlow backend.
W0623 09:59:26.631903  7648 deprecation_wrapper.py:119] From Q:\Program Files\Anaconda3\lib\site-packages\keras-2.2.4-py3.6.egg\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0623 09:59:26.876796  7648 deprecation_wrapper.py:119] From Q:\Program Files\Anaconda3\lib\site-packages\keras-2.2.4-py3.6.egg\keras\backend\tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0623 09:59:26.948606  7648 deprecation_wrapper.py:119] From Q:\Program Files\Anaconda3\lib\site-packages\keras-2.2.4-py3.6.egg\keras\backend\tensorflow_backend.py:4420: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0623 09:59:27.071277  7648 deprecation_wrapper.py:119] From Q:\Program Files\Anaconda3\lib\site-packages\keras-2.2.4-py3.6.egg\keras\backend\tensorflow_backend.py:4255: The name tf.nn.max_pool is depreca

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 158, 32)       896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 79, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 77, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 38, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 36, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 18, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 16, 128)       

In [ ]:
from screen_consts import WIDTH, HEIGHT, get_coordinates_from_direction
from grabscreen import grab_screen
from image_preproc import preproc_img
from getkeys import key_check
from image_preproc import prepare_image
import win32api, win32con
import time
import numpy as np
import cv2

def mouse_down(x,y):
    win32api.SetCursorPos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, x, y, 0, 0)

def mouse_up(x, y):
    win32api.SetCursorPos((x, y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, x, y, 0, 0)

    
from PIL import ImageGrab
def get_image():
    img = np.array(ImageGrab.grab())
    img = preproc_img(img)
    return img

def detect_lose(img): # input RGB
    img = roi(img, [np.array([[WIDTH/2-12, HEIGHT/2-5], [WIDTH/2+12, HEIGHT/2-5], [WIDTH/2+12, HEIGHT/2+3], [WIDTH/2-12, HEIGHT/2+3]], np.int32)])
    b = np.average(np.array(img[:,:,2]))
    g = np.average(np.array(img[:,:,1]))
    r = np.average(np.array(img[:,:,0]))
    return r < 0.6 and g > 1.5 and b > 0.5

def reload_page():
    win32api.keybd_event(win32con.VK_F5, 0, win32con.KEYEVENTF_EXTENDEDKEY, 0)  # press
    win32api.Sleep(50)
    win32api.keybd_event(win32con.VK_F5, 0, win32con.KEYEVENTF_EXTENDEDKEY | win32con.KEYEVENTF_KEYUP, 0)  # release


class_number = 12
img = grab_screen()
scr_W, scr_H = len(img[0]), len(img)


print("Start/Pause action with Alt+T")
paused = True
prev_img = get_image().reshape(-1, input_height, input_width, 3)
img = 0
still_image_counter = 0
while True:
    clock_prev = time.clock()
    keys = key_check()
    if "ALT" in keys and "T" in keys:
        paused ^= 1
        if paused:
            print("Stop action, press Alt+T to continue")
        else:
            print("Start action in 3 sec.")
            for i in list(range(3))[::-1]:
                time.sleep(1)
                print(i + 1)
            print("action!")
        time.sleep(1)
        
    if not paused:
        img = get_image().reshape(-1, input_height, input_width, 3)
        
        if np.sum(prev_img[0][:40]) == np.sum(img[0][:40]):
            still_image_counter += 1
        else:
            still_image_counter = 0

        if "C" in keys or still_image_counter > 3:
            offset = -55
            mouse_up(scr_W//2, scr_H//2+offset)
            mouse_down(scr_W//2, scr_H//2+offset)
            mouse_up(scr_W//2, scr_H//2+offset)
            
        # Refresh attempt in case of still image
        if still_image_counter == 60:
            print('frozen', int(time.clock()))
            ImageGrab.grab().save(f'recording_log/frozen-{int(time.clock())}.jpg')
            reload_page()
            time.sleep(10)
            still_image_counter = 0
            
        prev_img = img.copy()
        img = img / 255
        
        prediction_raw = model.predict(img)
        prediction = np.argmax(prediction_raw)
        x, y = get_coordinates_from_direction(prediction, scr_W, scr_H, class_number, radius=0.9)
        mouse_up(x, y)
        mouse_down(x, y)
        
        #print(np.round(model.predict(prep_img), 2), prediction)
        #print('Took {} ms'.format(round((time.clock() - clock_prev)*1000)))

Start/Pause action with Alt+T
Start action in 3 sec.
3
2
1
action!
frozen 7652
frozen 11224
frozen 11237
frozen 23013
frozen 23026
frozen 23527
frozen 34247
frozen 34260
frozen 34273
frozen 34286
frozen 47729
frozen 61288
frozen 75597
frozen 89484
Stop action, press Alt+T to continue
Start action in 3 sec.
3
2
1
action!
frozen 104095
frozen 109931
frozen 109945
frozen 109958
frozen 109971
frozen 109985
frozen 109998
frozen 110011
frozen 110024
frozen 110038
frozen 110051
frozen 110064
frozen 110077
frozen 110090
frozen 110103
frozen 110116
frozen 110129
frozen 110142
frozen 110156
frozen 110169
frozen 110182
frozen 110195
frozen 110208
frozen 110221
Stop action, press Alt+T to continue
Start action in 3 sec.
3
2
1
action!
frozen 116968
frozen 130466
frozen 132680
frozen 133096
frozen 148710
frozen 159573
frozen 174299
frozen 178646
frozen 178660
frozen 178674
Stop action, press Alt+T to continue
Start action in 3 sec.
3
2
1
action!
frozen 193935
frozen 207986
frozen 220047
frozen 23371

# VIEW INPUT

In [ ]:
%matplotlib notebook
import cv2
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def grab_frame():
    return get_image()

#create two subplots
gridsize = (2, 1)
fig = plt.figure(figsize=(6, 8))
ax1 = plt.subplot2grid(gridsize, (0, 0))

#create two image plots
im1 = ax1.imshow(grab_frame(), cmap='Greys_r')

def update(i):    
    im1.set_data(grab_frame())

ani = FuncAnimation(plt.gcf(), update, interval=10)
plt.show()

In [ ]:
np.sum(prev_img)

In [ ]:
import PIL
PIL.Image.fromarray(prev_img[0][:40])